# Создаём точку входа в Spark

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Можно использовать только rdd.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [6]:
def beautiful_print(rdd, num, title):
    print(title)
    for row in rdd.take(num):
        print('\t', row)

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_3')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 4)\
        .config('spark.executor.memory', f'{int(3000/1.1)}mb')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [48]:
train = sc.textFile('train.csv')
test = sc.textFile('test.csv')

In [49]:
# Выведите содержимое каждого файла (часть строк) (1 балл)

beautiful_print(train, 10, 'Train RDD')
beautiful_print(test, 10, 'Test RDD')

Train RDD
	 battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
	 842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
	 1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
	 563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
	 615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
	 1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1
	 1859,0,0.5,1,3,0,22,0.7,164,1,7,1004,1654,1067,17,1,10,1,0,0,1
	 1821,0,1.7,0,4,1,10,0.8,139,8,10,381,1018,3220,13,8,18,1,0,1,3
	 1954,0,0.5,1,0,0,24,0.8,187,4,0,512,1149,700,16,3,5,1,1,1,0
	 1445,1,0.5,0,0,0,53,0.7,174,7,14,386,836,1099,17,1,20,1,0,0,0
Test RDD
	 id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
	 1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
	 2,841,1,0.5

##  Преобразуем train и test

In [50]:
train_header = train.first()
test_header = test.first()

train = train\
    .filter(lambda row: row != train_header)\
    .map(lambda row: [float(el) for el in row.split(',')])


In [51]:
# Преобразуйте test (1 балл)

test = test\
    .filter(lambda row: row != test_header)\
    .map(lambda row: [float(el) for el in row.split(',')])

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)

PS: нужно сделать средсвтвами rdd pd.concat([train, test,], axis=0)

In [31]:
train.collect().__len__(), test.collect().__len__()

(2000, 1000)

In [131]:
# Объедините train и test (2 балла)
train_test_union = train.union(test)
train_test_union.collect().__len__()
train_test_union.toDF().toPandas()

,_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,...,_12,_13,_14,_15,_16,_17,_18,_19,_20,_21
0,842.0,0.0,2.2,0.0,1.0,0.0,7.0,0.6,188.0,2.0,...,20.0,756.0,2549.0,9.0,7.0,19.0,0.0,0.0,1.0,1.0
1,1021.0,1.0,0.5,1.0,0.0,1.0,53.0,0.7,136.0,3.0,...,905.0,1988.0,2631.0,17.0,3.0,7.0,1.0,1.0,0.0,2.0
2,563.0,1.0,0.5,1.0,2.0,1.0,41.0,0.9,145.0,5.0,...,1263.0,1716.0,2603.0,11.0,2.0,9.0,1.0,1.0,0.0,2.0
3,615.0,1.0,2.5,0.0,0.0,0.0,10.0,0.8,131.0,6.0,...,1216.0,1786.0,2769.0,16.0,8.0,11.0,1.0,0.0,0.0,2.0
4,1821.0,1.0,1.2,0.0,13.0,1.0,44.0,0.6,141.0,2.0,...,1208.0,1212.0,1411.0,8.0,2.0,15.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,996.0,1700.0,1.0,1.9,0.0,0.0,1.0,54.0,0.5,170.0,...,17.0,644.0,913.0,2121.0,14.0,8.0,15.0,1.0,1.0,0.0
2996,997.0,609.0,0.0,1.8,1.0,0.0,0.0,13.0,0.9,186.0,...,2.0,1152.0,1632.0,1933.0,8.0,1.0,19.0,0.0,1.0,1.0
2997,998.0,1185.0,0.0,1.4,0.0,1.0,1.0,8.0,0.5,80.0,...,12.0,477.0,825.0,1223.0,5.0,0.0,14.0,1.0,0.0,0.0
2998,999.0,1533.0,1.0,0.5,1.0,0.0,0.0,50.0,0.4,171.0,...,12.0,38.0,832.0,2509.0,15.0,11.0,6.0,0.0,1.0,0.0


## Рассчитайте соотношение сторон телефона и экрана

In [146]:
# Выведите отсортированное распределение соотношений сторон экрана(1 балла)
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)
# sc_h, sc_w

height = train_header.split(',').index('sc_h')
width = train_header.split(',').index('sc_w')

screen_wh_relation = train_test_union.map(lambda row: (row[width], row[height]))\
                                     .reduceByKey(lambda x, y: y / x)\
                                     .sortByKey()\
                                     .values()\
                                     .collect()

In [145]:
# Выведите отсортированное распределение плотности пикселей (1 балл)
# px_height, px_width, 1in ~ 96px

height = train_header.split(',').index('px_height')
width = train_header.split(',').index('px_width')
# как-то здесь не так. но я не понимаю, откуда взять диагональ
# перевести пиксели высоты и ширины в дюймы и высчитать по теореме Пифагора?
train_test_union.map(lambda row: (row[width], row[height]))\
                .reduceByKey(lambda x, y: (x**2 + y**2)**0.5/((y/64)**2 + (x/64)**2))\
                .sortByKey()\
                .collect()[:10]

[(0.0, 234.15273491457205),
 (2.0, 7.0),
 (3.0, 7.0),
 (4.0, 9.0),
 (5.0, 9.0),
 (6.0, 7.0),
 (7.0, 19.722854275552248),
 (9.0, 164.8987509203916),
 (10.0, 301.14390920962364),
 (11.0, 2.0)]

# JOIN
Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)

Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})

Имя функции |	Purpose |	Example |	Result
------------- |	------- |	------- |	------
subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}
join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}
rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}
cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}


In [ ]:
# 2 балла
###############
# Ваш код здесь
###############

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

In [160]:
# Считываем и объединяем данные (1 балл)
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)

train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test.csv', header=True, inferSchema=True)

# Создаём колонки с соотношением сторон и плотностью пикселей (1 балл)


## Сохранение


In [ ]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора в формате "1day 1hour 1minute"
# 2 балла

